# Dataset generation

In [1]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as img
import numpy as np
import cv2
import os, shutil
from scipy.spatial import distance

In [2]:
# 각 행의 n 번째 열을 기준으로 sort (오름차순)
# x : nd array
# axis : 기준 열
def sort_ndarray_by_col(x, axis=0):
    size = x.shape[0]
    col = [x[i][axis] for i in range(size)] # 기준 행의 원소들을 추출
    sorted_col = sorted(col) # 추출된 원소들을 정렬
    rank = [sorted_col.index(x[i][axis]) for i in range(size)] # 정렬된 index 추출
    #print(rank)
    try:
        sorted_array = [x[rank.index(i)] for i in range(size)]  # 정렬된 index 순서대로 정렬
    except: # 똑같은 순위가 있는 경우 예외처리
        overlap_rank = []
        for i in range(size):
            if rank.count(i) > 1:
                overlap_rank.append(i)
        for r in overlap_rank:
            overlap_list = [i for i in range(size) if rank[i]==r]
            for k, idx in enumerate(overlap_list):
                rank[idx] += k
        sorted_array = [x[rank.index(i)] for i in range(size)]  # 정렬된 index 순서대로 정렬

    return np.array(sorted_array)

# 함수화
def perspective_transform(src, point, height=256, width=256):    
    # 4 꼭지점 분류 
    sorted_by_y= sort_ndarray_by_col(point, 1)
    top_two = sorted_by_y[0:2]
    topLeft, topRight = sort_ndarray_by_col(top_two, 0)
    bottom_two = sorted_by_y[2:4]
    bottomLeft, bottomRight = sort_ndarray_by_col(bottom_two, 0)
    #print("tl:", topLeft)
    #print("bl", bottomLeft)
    #print("tr", topRight)
    #print("br", bottomRight)
    
    
    # 변환 전 4개 좌표 
    srcPoint = np.float32([topLeft, topRight, bottomRight , bottomLeft])

    # 변환 후 4개 좌표
    dstPoint = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype=np.float32)

    # Perspective transformation
    matrix = cv2.getPerspectiveTransform(srcPoint, dstPoint)
    dst = cv2.warpPerspective(src, matrix, (width, height))
    
    return dst

# 이미지 상에 polygon 그리기
def draw_label(img_path, data, save_path=""):
    '''
    image = img.imread(img_path)
    plt.imshow(image)

    ax = plt.gca()

    point = np.array([[data['p1'][0], data['p1'][1]],
                      [data['p2'][0], data['p2'][1]],
                      [data['p3'][0], data['p3'][1]],
                      [data['p4'][0], data['p4'][1]]])
    #print(point)

    polygon = patches.Polygon(point, fill=None ,edgecolor='k',ls='solid',lw=3)

    ax.add_patch(polygon)

    plt.show()
    '''
    src = cv2.imread(img_path, cv2.IMREAD_COLOR)
    point = np.array([[data['b1'][0], data['b1'][1]],
                      [data['b2'][0], data['b2'][1]],
                      [data['b3'][0], data['b3'][1]],
                      [data['b4'][0], data['b4'][1]]], dtype=np.float32)
    
    dst = cv2.polylines(src, np.int32([point]), True, (255, 0, 0), 6)

    return dst

In [3]:
import os 

path = './'
file_list = os.listdir(path) 

print (file_list)

['test', '.ipynb_checkpoints', 'custom', 'generate_dataset.ipynb', 'open']


In [4]:
# xml파일로부터 parsing
import xml.etree.ElementTree as ET

dataset_list = ['test']

datasets = {}

for dataset_name in dataset_list:
    
    # Read annotation file
    filePath = "./"+dataset_name+"/annotations.xml"
    tree = ET.parse(filePath)
    root = tree.getroot()

    dataset = {}
    empty = 0
    car = 0

    for child in root:
        if 'image' in child.tag: # image tag 찾기 
            # Read attribution : name, width, height
            name = child.attrib['name']
            width = child.attrib['width']
            height = child.attrib['height']
            
            for children in child.getchildren(): # 각 image tag 내 labeling 중
                label = children.attrib['label']
                if label not in ["있음", "없음"]: # labeling이 있음/없음이 아니면 pass
                    continue
                try:
                    points = children.attrib['points'] # labeling이 있음 또는 없음 인 경우 points 정보 저장
                except:
                    continue
                if len(points.split(';')) == 5: # 라벨링된 point가 5개인 경우
                    #print(points.split(';'))
                    #print(name)
                    #print(points)
                    ps = [(float(points.split(';')[0].split(',')[0]), float(points.split(';')[0].split(',')[1])),
                         (float(points.split(';')[1].split(',')[0]), float(points.split(';')[1].split(',')[1])),
                         (float(points.split(';')[2].split(',')[0]), float(points.split(';')[2].split(',')[1])),
                         (float(points.split(';')[3].split(',')[0]), float(points.split(';')[3].split(',')[1])),
                         (float(points.split(';')[4].split(',')[0]), float(points.split(';')[4].split(',')[1]))]
                    left = []
                    right = []
                    up = []
                    down = []
                    for i, p in enumerate(ps):
                        if p[0] == 0:  
                            print(i,p)
                            left.append(i)
                        if p[0] == float(width):
                            print(i,p)
                            right.append(i)
                        if p[1] == 0:
                            print(i,p)
                            up.append(i)
                        if p[1] == float(height):
                            print(i,p)
                            down.append(i)
                    #print(len(left), len(right), len(up), len(down))
                    
                    if len(left) == 2:
                        del ps[left[0]]
                    elif len(right) == 2:
                        del ps[right[0]]
                    elif len(up) == 2:
                        del ps[up[0]]
                    elif len(down) == 2:
                        del ps[down[0]]
                    #print(ps, "\n")
                    
                elif len(points.split(';')) == 4: # 라벨링된 point가 4개인 경우 = polygon 라벨링 (없음)
                    ps = [(float(points.split(';')[0].split(',')[0]), float(points.split(';')[0].split(',')[1])),
                         (float(points.split(';')[1].split(',')[0]), float(points.split(';')[1].split(',')[1])),
                         (float(points.split(';')[2].split(',')[0]), float(points.split(';')[2].split(',')[1])),
                         (float(points.split(';')[3].split(',')[0]), float(points.split(';')[3].split(',')[1]))]
                    
                    # 4 꼭지점 분류 
                    sorted_by_y= sort_ndarray_by_col(np.array(ps), 1)
                    top_two = sorted_by_y[0:2]
                    topLeft, topRight = sort_ndarray_by_col(top_two, 0)
                    bottom_two = sorted_by_y[2:4]
                    bottomLeft, bottomRight = list(sort_ndarray_by_col(bottom_two, 0))
                    
                elif len(points.split(';')) == 2: # 라벨링된 point가 2개인 경우 = 직선 라벨링 (있음))
                    ps = [(float(points.split(';')[0].split(',')[0]), float(points.split(';')[0].split(',')[1])),
                         (float(points.split(';')[1].split(',')[0]), float(points.split(';')[1].split(',')[1]))]
                    
                    # 2 꼭지점 분류
                    if ps[0][0] < ps[1][0]:
                        bottomLeft = ps[0]
                        bottomRight = ps[1]
                    else:
                        bottomLeft = ps[1]
                        bottomRight = ps[0]
                    
                else:
                    continue
                print('bb', bottomLeft, bottomRight) # bounding box 생성에 기준이 되는 주차면 앞 라인의 두 점
                    
                # 라인으로부터 바운딩 박스 생성
                x_offset = 0
                width = distance.euclidean(bottomLeft,bottomRight)+x_offset
                y_offset = width/10.0
                height = -width*2/3.0
                print(ps, width, height)
                    

                b1 = (bottomLeft[0], bottomLeft[1]+y_offset)
                b2 = (bottomRight[0], bottomRight[1]+y_offset)
                b3 = (b2[0], b2[1]+height)
                b4 = (b1[0], b1[1]+height)
                    
                # 추출한 bounding box 정보를 dictionary에 저장
                if children.attrib['label'] == '있음':
                    car += 1
                    dataset['car'+str(car)] = {'label':label,
                                               'src':name, 
                                               'b1':b1,
                                               'b2':b2,
                                               'b3':b3,
                                               'b4':b4}
                elif children.attrib['label'] == '없음':
                    empty +=1
                    dataset['empty'+str(empty)] = {'label':label,
                                               'src':name, 
                                               'p1':ps[0], # p1~4 : 주차면 polygon 4 point
                                               'p2':ps[1],
                                               'p3':ps[2],
                                               'p4':ps[3],
                                               'b1':b1, # b1~4 : bounding box 4 point
                                               'b2':b2,
                                               'b3':b3,
                                               'b4':b4}
    # 각 폴더마다 추출된 정보를 저장
    datasets[dataset_name] = dataset

bb (1055.77, 747.92) (1387.67, 720.37)
[(1055.77, 747.92), (1387.67, 720.37)] 333.04145762952703 -222.02763841968468
bb (731.39, 756.69) (1054.52, 746.67)
[(731.39, 756.69), (1054.52, 746.67)] 323.2853187201671 -215.5235458134447
bb [454.6  751.68] [728.88 756.69]
[(454.6, 751.68), (728.88, 756.69), (849.1, 578.8), (710.1, 590.11)] 274.32575252790247 -182.88383501860164


/tmp/ipykernel_16/3016376879.py:26: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  for children in child.getchildren(): # 각 image tag 내 labeling 중


In [5]:
datasets.keys()

dict_keys(['test'])

In [6]:
# test print
print(dataset_list[0])
datasets[dataset_list[0]]

test


{'car1': {'label': '있음',
  'src': 'VID_20220724_143757_00_008_2022-08-02_00-22-13_스크린샷.jpg',
  'b1': (1055.77, 781.2241457629526),
  'b2': (1387.67, 753.6741457629527),
  'b3': (1387.67, 531.646507343268),
  'b4': (1055.77, 559.196507343268)},
 'car2': {'label': '있음',
  'src': 'VID_20220724_143757_00_008_2022-08-02_00-22-13_스크린샷.jpg',
  'b1': (731.39, 789.0185318720168),
  'b2': (1054.52, 778.9985318720167),
  'b3': (1054.52, 563.474986058572),
  'b4': (731.39, 573.4949860585721)},
 'empty1': {'label': '없음',
  'src': 'VID_20220724_143757_00_008_2022-08-02_00-22-13_스크린샷.jpg',
  'p1': (454.6, 751.68),
  'p2': (728.88, 756.69),
  'p3': (849.1, 578.8),
  'p4': (710.1, 590.11),
  'b1': (454.6, 779.1125752527902),
  'b2': (728.88, 784.1225752527903),
  'b3': (728.88, 601.2387402341886),
  'b4': (454.6, 596.2287402341885)}}

In [9]:
for dataset_name in dataset_list:
    
    dataset_name_replaced = dataset_name.replace('-', '_')
    print(dataset_name, dataset_name_replaced)
    
    FOLDER_NAME = dataset_name # 생성된 patch가 저장될 폴더 이름
    DATASET_ROOT = "/root/share/datasets/result" # root folder 
    SPLIT_PATH = "/root/share/pkgs/deep-parking_cpu/splits/"
    
    DATASET_PATH = DATASET_ROOT + FOLDER_NAME
    EMPTY_PATH = DATASET_PATH + "/Empty/" # 없음 patch 저장 폴더
    OCCUPIED_PATH = DATASET_PATH + "/Occupied/" # 있음 patch 저장 폴더
    SEGMENTED_PATH = DATASET_PATH + "/Segmented/" # 주차면 polygon perspective transformation한 patch 저장 폴더
    print(DATASET_PATH)
    print(EMPTY_PATH)
    print(OCCUPIED_PATH)

    # 기존 폴더 삭제
    if os.path.exists(DATASET_PATH):
        shutil.rmtree(DATASET_PATH)
    
    # 폴더가 없는 경우 생성
    if not os.path.isdir(DATASET_PATH):
        os.makedirs(DATASET_PATH)
    if not os.path.isdir(EMPTY_PATH):
        os.makedirs(EMPTY_PATH)
    if not os.path.isdir(OCCUPIED_PATH):
        os.makedirs(OCCUPIED_PATH)
    if not os.path.isdir(SEGMENTED_PATH):
        os.makedirs(SEGMENTED_PATH)

    # Open label txt file 
    label_file_all = open(DATASET_PATH + '/'+ dataset_name_replaced + '_all.txt', 'w')
    label_file_occupied = open(DATASET_PATH + '/' + dataset_name_replaced + '_occupied.txt', 'w')
    label_file_empty = open(DATASET_PATH + '/' + dataset_name_replaced + '_empty.txt', 'w')
    label_file_segmented = open(DATASET_PATH + '/' + dataset_name_replaced + '_segmented.txt', 'w')
    
    # save path for labeled image
    LABELED_PATH = './'+dataset_name+'/labeled_data/'
    print(LABELED_PATH)
    if os.path.exists(LABELED_PATH):
        shutil.rmtree(LABELED_PATH)
    if not os.path.isdir(LABELED_PATH):
        os.makedirs(LABELED_PATH)
        
    dataset = datasets[dataset_name]
    
    for key in dataset.keys():
        print(key)
        data = dataset[key] # key : 'label', 'src', 'p1', 'p2', 'p3', 'p4', 'b1', 'b2', 'b3', 'b4'
        label = data['label'] # empty or car
        print(data)
        
        bb_point = np.array([[data['b1'][0], data['b1'][1]],
                            [data['b2'][0], data['b2'][1]],
                            [data['b3'][0], data['b3'][1]],
                            [data['b4'][0], data['b4'][1]]], dtype=np.float32)
        
        # source image
        img_root = './'+dataset_name+'/images/'
        img_path = img_root + data['src']
        print(img_path)
        src = cv2.imread(img_path, cv2.IMREAD_COLOR)

        # labeled image
        labeled_img = draw_label(img_path, data)
        #print(labeled_img)
        
        # save labeld image
        cv2.imwrite(LABELED_PATH+data['src'].split('.')[0]+'_'+key+'_with_label.jpg', labeled_img)
        #plt.imshow(labeled_img)
        #plt.show()

        # cropped patch
        dst = perspective_transform(src, bb_point, 256, 256)
        
        # save cropped patch
        img_name = data['src'].split('.')[0]+'_'+key+'.jpg'
        cv2.imwrite(LABELED_PATH+img_name, dst)
        #plt.imshow(dst)
        #plt.show()
        
        if label == '없음':
            # save empty patch
            cv2.imwrite(EMPTY_PATH+img_name, dst)
            label_file_all.write(FOLDER_NAME+'/Empty/'+img_name+' 0\n')
            label_file_empty.write(FOLDER_NAME+'/Empty/'+img_name+' 0\n')
            
            
            # segmented patch 
            seg_point = np.array([[data['p1'][0], data['p1'][1]],
                          [data['p2'][0], data['p2'][1]],
                          [data['p3'][0], data['p3'][1]],
                          [data['p4'][0], data['p4'][1]]], dtype=np.float32)
            dst = perspective_transform(src, seg_point, 256, 256)
            cv2.imwrite(SEGMENTED_PATH+img_name, dst)
            #plt.imshow(dst)
            #plt.show()
            
            # save segmented patch 
            label_file_segmented.write(img_name+' '
                                       +str(seg_point[0])+' '
                                       +str(seg_point[1])+' '
                                       +str(seg_point[2])+' '
                                       +str(seg_point[3])+' '
                                       +'0\n')     
            
        elif label == '있음':
            # save occupied patch
            cv2.imwrite(OCCUPIED_PATH+img_name, dst)
            label_file_all.write(FOLDER_NAME+'/Occupied/'+img_name+' 1\n')
            label_file_occupied.write(FOLDER_NAME+'/Occupied/'+img_name+' 1\n')
            
        #plt.imshow(dst)
        #plt.show()
        
    # close label text files
    label_file_all.close()
    label_file_occupied.close()
    label_file_empty.close()
    label_file_segmented.close()
    
    #shutil.copy(DATASET_PATH + '/'+ dataset_name_replaced + '_all.txt', SPLIT_PATH) 
    #shutil.copy(DATASET_PATH + '/'+ dataset_name_replaced + '_occupied.txt', SPLIT_PATH) 
    #shutil.copy(DATASET_PATH + '/'+ dataset_name_replaced + '_empty.txt', SPLIT_PATH) 
    

test test
/root/share/datasets/custom/dataset/Bluecom/test
/root/share/datasets/custom/dataset/Bluecom/test/Empty/
/root/share/datasets/custom/dataset/Bluecom/test/Occupied/
./test/labeled_data/
car1
{'label': '있음', 'src': 'VID_20220724_143757_00_008_2022-08-02_00-22-13_스크린샷.jpg', 'b1': (1055.77, 781.2241457629526), 'b2': (1387.67, 753.6741457629527), 'b3': (1387.67, 531.646507343268), 'b4': (1055.77, 559.196507343268)}
./test/images/VID_20220724_143757_00_008_2022-08-02_00-22-13_스크린샷.jpg


[ WARN:0@140.652] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('./test/images/VID_20220724_143757_00_008_2022-08-02_00-22-13_스크린샷.jpg'): can't open/read file: check file path/integrity
[ WARN:0@140.652] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('./test/images/VID_20220724_143757_00_008_2022-08-02_00-22-13_스크린샷.jpg'): can't open/read file: check file path/integrity


error: OpenCV(4.6.0) /io/opencv/modules/imgcodecs/src/loadsave.cpp:801: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'
